# Example

## Bands WorkGraph

In [2]:
from workgraph_collections.qe.bands import bands_workgraph
from aiida import load_profile
load_profile()

bands_wg = bands_workgraph(run_relax=True)
bands_wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

## PDOS WorkGraph

In [3]:
from workgraph_collections.qe.pdos import pdos_workgraph
pdos_wg = pdos_workgraph(run_scf=True)
pdos_wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

## Node Groph
Here is an example to combine bands workgraph, pdos workgraph with a relax node to form a new workgraph.

In [4]:
from aiida_workgraph import WorkGraph
from aiida_quantumespresso.workflows.pw.relax import PwRelaxWorkChain

wg = WorkGraph('Electronic Structure')
relax_node = wg.nodes.new(PwRelaxWorkChain, name='relax')
bands_job = wg.nodes.new(bands_workgraph, name='bands_group')
pdos_job = wg.nodes.new(pdos_workgraph, name='pdos_group')
wg.links.new(relax_node.outputs['output_structure'], bands_job.inputs['structure'])
wg.links.new(relax_node.outputs['output_structure'], pdos_job.inputs['structure'])
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'Electro…

## Append WorkGraph
First, create a workgraph with relax node:

In [5]:
from aiida_workgraph import WorkGraph
from aiida_quantumespresso.workflows.pw.relax import PwRelaxWorkChain

wg = WorkGraph('Electronic Structure')
relax_node = wg.nodes.new(PwRelaxWorkChain, name='relax')
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'Electro…

Now let’s combine bands workgraph, pdos workgraph with a relax node to form a new workgraph.

To append a workgraph into another workgraph, there are two steps: - append the workgraph - adjust the links, add new links between nodes, and remove old links if needed.

In [6]:
# append a wroktree
wg.append(bands_wg, prefix='bands_')
wg.append(pdos_wg, prefix='pdos_')
# adjust the links
wg.links.new(relax_node.outputs['output_structure'], wg.nodes['bands_scf'].inputs['pw.structure'])
wg.links.new(relax_node.outputs['output_structure'], wg.nodes['pdos_scf'].inputs['pw.structure'])
wg

NodeGraphWidget(positions={'relax': [30, 30]}, settings={'minimap': True}, style={'width': '90%', 'height': '6…

## Base Relax WorkGraph

In [19]:
# -*- coding: utf-8 -*-
"""Test."""
from copy import deepcopy

from aiida import load_profile
from aiida.orm import Dict, KpointsData, StructureData, load_code, load_group
from ase.build import bulk

from workgraph_collections.qe.relax import relax_workgraph

load_profile()

atoms = bulk('Si')
structure_si = StructureData(ase=atoms)

code = load_code('qe-7.2-pw@localhost')
paras = Dict({
    'CONTROL': {
        'calculation': 'scf',
    },
    'SYSTEM': {
        'ecutwfc': 30,
        'ecutrho': 240,
        'occupations': 'smearing',
        'smearing': 'gaussian',
        'degauss': 0.1,
    },
})
relax_paras = deepcopy(paras)
relax_paras.get_dict()['CONTROL']['calculation'] = 'vc-relax'

kpoints = KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
# Load the pseudopotential family.
pseudo_family = load_group('SSSP/1.3/PBEsol/efficiency')
pseudos = pseudo_family.get_pseudos(structure=structure_si)
#
metadata = {
    'options': {
        'resources': {
            'num_machines': 1,
            'num_mpiprocs_per_machine': 1,
        },
    }
}

relax_inputs = {
    'pw': {
        'code': code,
        'pseudos': pseudos,
        'parameters': relax_paras,
        'metadata': metadata,
    },
    'kpoints': kpoints,
}

wg = relax_workgraph(
    structure=structure_si,
    inputs=relax_inputs,
    max_iterations=5,
)
display(wg)
wg.submit(wait=True)


NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

WorkGraph node created, PK: 4637


'FINISHED'